In [1]:
import cv2 
import imageio
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random
import seaborn as sns; sns.set()
import tensorflow as tf

# Problem:
This is a ***binary classification supervised learning problem*** where the data has labels. The main goal is to predict whether an image shows metastastic tissue or not. 

# Exploratory Data Analysis 

In [2]:
traindf=pd.read_csv("/kaggle/input/histopathologic-cancer-detection/train_labels.csv")
traindf.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [4]:
print('Train shape:', traindf.shape)

Train shape: (220025, 2)


In [8]:
print('Train columns:', traindf.columns.tolist())

Train columns: ['id', 'label']


## Format of Sample Submission

In [11]:
sample_submission = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/sample_submission.csv')
print(sample_submission.head())

                                         id  label
0  0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5      0
1  95596b92e5066c5c52466c90b69ff089b39f2737      0
2  248e6738860e2ebcf6258cdc1f32f299e0c76914      0
3  2c35657e312966e9294eac6841726ff3a748febf      0
4  145782eb7caa1c516acbe2eda34d9a3f31c41fd6      0


## Check for Class Imbalance

In [ ]:
# sanity check that there are two classes 
traindf['label'].unique()

In [ ]:
num_imgs = len(traindf)
print('Image number of training images:', num_imgs)

In [ ]:
# check to see if there is class imbalance 
labelCounts = traindf['label'].value_counts()
print(labelCounts)

In [ ]:
# get the percentages 
traindf['label'].value_counts(normalize=True)

In [ ]:
# histogram plotting function with KDE overlay 
# plot mean as a red line 
def hist_plot(x, df):
    count = df[x].value_counts()
    sns.barplot(count.index, count.values, alpha=0.8)
    plt.title("Histogram of Classes", fontsize=18)
    plt.xlabel(x, fontsize=16)
    plt.ylabel('Frequency', fontsize=16)
    plt.show();

hist_plot("label", traindf)

There is about a 60% to 40% split of 0 labels to 1 labels. The **majority class** is no tumor tissue. 
This Google Developers' course indicates that if the minority class is 40% of the dataset that is a mild degree of imbalance. The recommendation for an imbalanced dataset is to first try training on the true distribution and see if the model works well and generalizes well. If it doesn't, techniques like downsampling or upweighting can be used. 
Source: https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data

## Show Sample Images with Labels 

In [ ]:
def append_ext(fn):
    return fn+".tif"
""""
the id is the same as the filename without the .tif extension
add the .tif extension
this will make it possible to use datagen.flow_from_dataframe 
to create the datasets 
"""
traindf["id"]=traindf["id"].apply(append_ext)
traindf.head()

In [ ]:
# the training labels are the sample_submission.csv file 
testdf=pd.read_csv("/kaggle/input/histopathologic-cancer-detection/sample_submission.csv",dtype=str)
testdf["id"]=testdf["id"].apply(append_ext)
testdf.head()

In [ ]:
data = plt.imread('/kaggle/input/histopathologic-cancer-detection/train/' + traindf["id"].iloc[0])
print('Shape of image array:', data.shape)

The images are 96 x 96 pixels with 3 R,G, B color channels. 

In [ ]:
# print image data type 
print('Image type:', data.dtype)

In [ ]:
# display memory usage of image 
# in kilobytes 
print('Memory usage:', data.size, 'KB') 

If all values in an image are positive, unsigned integers can cover the largest range of values while taking up the least amount of memory. 

## Image Metadata


In [ ]:
# load dictionary of available metadata 
im = imageio.imread('/kaggle/input/histopathologic-cancer-detection/train/' + traindf["id"].iloc[0])
im.meta

Some image metadata contains information like patient demographics. In this case, the available metadata is limited to information like shape, datetime, and no compression. 

## Display random sampling of images with labels.

In [ ]:
def random_sample(df, num_samples):
    """
    takes in dataframe 
    and number of samples 
    returns tuple with array of images,
    array of labels for those images
    """
    number_train_images = len(df)
    
    randomlist = random.sample(range(number_train_images), 
                               num_samples)
    imgs = []
    labels = []
    for i in randomlist:
        path = df.iloc[i]["id"]
        label = df.iloc[i]['label']
        imgs.append(path)
        labels.append(label)
    return (imgs, labels)

In [ ]:
def display_image(imgs, labels, num_samples):
    """
    takes in arrays of images and labels
    displays image with label 
    """
    # Displaying the loaded images
    fig = plt.figure(figsize=(10, 4), dpi=150)

    for i in range(len(imgs)): 
        plt.axis('off')
        ax = fig.add_subplot(1, num_samples, i+1, xticks=[], yticks=[])
        data = plt.imread('/kaggle/input/histopathologic-cancer-detection/train/' + imgs[i])
        # imshow displays 2D image data 
        ax.imshow(data)
        ax.set_title('Label: ' + str(labels[i])) 

In [ ]:
# filter out images with no tumor tissue
no_tumor_tissue = traindf[traindf['label']==0]
no_tumor_tissue.head()

In [ ]:
# filter out images with tumor tissue
tumor_tissue = traindf[traindf['label']==1]
tumor_tissue.head()

In [ ]:
num_samples = 5 
# display negative images 
imgs_neg, labels_neg = random_sample(no_tumor_tissue, num_samples)
display_image(imgs_neg, labels_neg, num_samples)
# display positive images 
imgs_pos, labels_pos = random_sample(tumor_tissue, num_samples)
display_image(imgs_pos, labels_pos, num_samples)

As someone who isn't a professional pathologist, I can't identify the differences in classes from looking at these images alone.

## Image Intensity Histogram
**Pixels** are 2D picture elements. They have two properties: intensity and location. <br>
Histograms summarize the distribution of intensity values in an image. They bin each pixel by its value.

In [ ]:
"""
this function is from this StackOverflow post:
https://stackoverflow.com/questions/55659784/plot-multiple-rgb-images-and-histogram-side-by-side-in-a-grid
"""
def histogram_img(img, title=None):
    plt.figure(figsize=(16,6))
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2)
    # cv2 is in B,G,R order
    colors = ('b','g','r')
    histograms = []

    for i in range(3): 
        hist = cv2.calcHist([img], [i], None, [256], [0, 255])
        histograms.append(hist)
        ax1.plot(hist, color=colors[i])
    ax2.imshow(img)
    ax2.grid(False)
    ax2.axis('off') 
    

    if title is not None:
        plt.title =  title
        plt.xlabel = "Bins"
        plt.ylabel = "# Pixels"

    plt.show()

In [ ]:
def image_path(randomlist, df):
    img_path = []
    labels = []
    for i in randomlist:
        path = df.iloc[i]["id"]
        label = df.iloc[i]['label']
        image = cv2.imread('/kaggle/input/histopathologic-cancer-detection/train/' + path)
        img_path.append(image)
        labels.append(label)
    return (img_path, labels)

## Flattened Color Histograms: No Tumor Tissue 

In [ ]:
#Generate random indices in the range of number of training images
randomlist = random.sample(range(len(no_tumor_tissue)), num_samples)
img_path, labels = image_path(randomlist, no_tumor_tissue)

In [ ]:
for i in range(len(img_path)):
    histogram_img(img_path[i])

## Flattened Color Histograms: Tumor Tissue

In [ ]:
#Generate random indices in the range of number of training images
randomlist = random.sample(range(len(tumor_tissue)), num_samples)
img_path, labels = image_path(randomlist, tumor_tissue)

for i in range(len(img_path)):
    histogram_img(img_path[i])

Another resource consulted: 
+ https://www.pyimagesearch.com/2014/01/22/clever-girl-a-guide-to-utilizing-color-histograms-for-computer-vision-and-image-search-engines/